In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image

# Load the sales data
sales_data = pd.read_csv(r"C:\Users\nandi\OneDrive\Desktop\sales_data.csv")

# Strip whitespace from column names
sales_data.columns = sales_data.columns.str.strip()

# Load the customer data
customer_data = pd.read_csv(r"C:\Users\nandi\OneDrive\Desktop\customer_data.csv")

# Strip whitespace from column names
customer_data.columns = customer_data.columns.str.strip()

# Merge the data
merged_data = pd.merge(sales_data, customer_data, on='CustomerID', how='inner')

# Strip whitespace from merged data column names
merged_data.columns = merged_data.columns.str.strip()

# Convert 'Date' to datetime format
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# Create a new Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Dashboard"

# Add the merged data to the Excel file
for r in dataframe_to_rows(merged_data, index=False, header=True):
    ws.append(r)

# Add a blank row for spacing
ws.append([])

# 1. Total Sales Over Time
sales_over_time = merged_data.groupby('Date')['Total_Cost'].sum().reset_index()
plt.figure(figsize=(10, 6))
plt.plot(sales_over_time['Date'], sales_over_time['Total_Cost'], marker='o', color='blue')
plt.title('Total Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('total_sales_over_time.png')
plt.close()

# Insert the Total Sales Over Time chart into the Excel file
img1 = Image('total_sales_over_time.png')
ws.add_image(img1, 'A10')  # Adjust the cell position as needed

# 2. Sales Distribution by Item
sales_by_item = merged_data.groupby('Item')['Total_Cost'].sum().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(sales_by_item['Item'], sales_by_item['Total_Cost'], color='green')
plt.title('Sales Distribution by Item')
plt.xlabel('Item')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('sales_distribution_by_item.png')
plt.close()

# Insert the Sales Distribution by Item chart into the Excel file
img2 = Image('sales_distribution_by_item.png')
ws.add_image(img2, 'A25')  # Adjust the cell position as needed

# 3. Sales by Customer Location
if 'location' in merged_data.columns:
    sales_by_location = merged_data.groupby('location')['Total_Cost'].sum().reset_index()
    plt.figure(figsize=(8, 8))
    plt.pie(sales_by_location['Total_Cost'], labels=sales_by_location['location'], autopct='%1.1f%%', startangle=140)
    plt.title('Sales Distribution by Customer Location')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.savefig('sales_by_location.png')
    plt.close()

    # Insert the Sales by Customer Location chart into the Excel file
    img3 = Image('sales_by_location.png')
    ws.add_image(img3, 'A40')  # Adjust the cell position as needed

# 4. Units Sold by Item
units_sold_by_item = merged_data.groupby('Item')['Units'].sum().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(units_sold_by_item['Item'], units_sold_by_item['Units'], color='orange')
plt.title('Units Sold by Item')
plt.xlabel('Item')
plt.ylabel('Total Units Sold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('units_sold_by_item.png')
plt.close()

# Insert the Units Sold by Item chart into the Excel file
img4 = Image('units_sold_by_item.png')
ws.add_image (img4, 'A55')  # Adjust the cell position as needed

# Save the workbook
wb.save('Sales_Dashboard.xlsx')